# Introduction: Fama / French Replication

Each year (usually at the end of June), portfolios are published to the Fama / French data library. These portfolios consist of numerous univariate and bivariate sorts, with a wide variety of factors. Analysis is then conducted on each of these sorts using CRPS and Compustat data. The resulting tables include monthly value and equal weighted returns, daily returns, annual returns, the monthly number of firms in each portfolio, and much more. Our mission is to replicate a small subset of those portfolios, using the brief descrpitions provided on the website for how each portfolio was structured.

# Task 1: Pulling Data from Ken French Website using `pandas_datareader`

The specific portfolios we are attemtping to replicate are the following:

* Bivariate (univariate) sorts formed on size and earnings/price, size and cashflow/price, and size and dividend yield
* Bivariate sorts formed on operating profitability and investment
* Each of the “5 industry portfolios” and “49 industry portfolios”

To begin, we pulled the final data from online so we could reference it throughout the process.

In [94]:
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
import sys
sys.path.insert(0, '../src')
from pull_test_data import *
import load_CRSP_Compustat

import pandas as pd
from pathlib import Path
import config
import warnings
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import zipfile
import csv
warnings.filterwarnings("ignore")

In [95]:
def save_portfolio_data_to_excel(portfolio_descriptions):
    """
    Fetches data for each portfolio and saves it to an Excel file, with descriptions and data in separate sheets.

    Args:
        portfolio_descriptions (dict): A dictionary containing portfolio names as keys and tuples of start and end dates as values.

    Returns:
        None
    """
    for portfolio_name, (start_date, end_date) in portfolio_descriptions.items():
        # Fetch data for the portfolio
        data = web.DataReader(portfolio_name, 'famafrench', start=start_date, end=end_date)
        
        # Define the Excel file path
        excel_path = filedir / f"{portfolio_name.replace('/', '_')}.xlsx"  # Ensure the name is file-path friendly
        with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
            # Write the description to the first sheet
            if 'DESCR' in data:
                description_df = pd.DataFrame([data['DESCR']], columns=['Description'])
                description_df.to_excel(writer, sheet_name='Description', index=False)
            
            # Write each table in the data to subsequent sheets
            for table_key, df in data.items():
                if table_key == 'DESCR':
                    continue  # Skip the description since it's already handled
                sheet_name = str(table_key)  # Naming sheets by their table_key
                df.to_excel(writer, sheet_name=sheet_name[:31])  # Sheet name limited to 31 characters

In [96]:
def scrape_and_download(base_url, filedir, portfolios):
    """
    Scrapes the webpage content from the given base URL, filters the relevant links,
    and downloads and extracts the ZIP files for the portfolios of interest.

    Args:
        base_url (str): The base URL of the webpage to scrape.
        filedir (str): The directory where the downloaded files will be saved.
        portfolios (list): A list of portfolio names to filter the links.

    Returns:
        None
    """
    # Fetch the webpage content
    response = requests.get(base_url)
    if response.status_code != 200:
        print(f"Failed to fetch {base_url}")
        return
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Filter the <a> tags to find relevant links
    links = soup.find_all('a', href=True)
    for link in links:
        href = link.get('href')
        if href and "CSV.zip" in href:
            # Check if the link matches any portfolios of interest
            portfolio_name = href.split("_CSV.zip")[0]
            if any(portfolio in portfolio_name for portfolio in portfolios):
                full_url = urljoin(base_url, href)
                print(f"Found: {full_url}")
                download_and_extract_zip(full_url, filedir, portfolio_name)

In [97]:
# if __name__ == "__main__":
#     save_portfolio_data_to_excel(portfolio_descriptions)
#     scrape_and_download(base_url, filedir, portfolios)

Upon completing this step, various excel files appeared within our /data folder. An example of the descriptions for each sheet of data for one set of portfolios is shown below.

In [98]:
sheet = pd.read_excel(f'../data/famafrench/5_Industry_Portfolios.xlsx', 'Description')
print(sheet['Description'][0])

5 Industry Portfolios
---------------------

This file was created by CMPT_IND_RETS using the 202401 CRSP database. It contains value- and equal-weighted returns for 5 industry portfolios. The portfolios are constructed at the end of June. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. Copyright 2024 Kenneth R. French

  0 : Average Value Weighted Returns -- Monthly (1170 rows x 5 cols)
  1 : Average Equal Weighted Returns -- Monthly (1170 rows x 5 cols)
  2 : Average Value Weighted Returns -- Annual (97 rows x 5 cols)
  3 : Average Equal Weighted Returns -- Annual (97 rows x 5 cols)
  4 : Number of Firms in Portfolios (1170 rows x 5 cols)
  5 : Average Firm Size (1170 rows x 5 cols)
  6 : Sum of BE / Sum of ME (98 rows x 5 cols)
  7 : Value-Weighted Average of BE/ME (98 rows x 5 cols)


# Task 2: Pulling raw data from CRSP and compustat

Once seeing what the final product should look like, we need to pull to raw data off the web to start our construction. The return data is gathered from CRSP, whereas much of the other relevant data needs to be merged from compustat.

In [99]:
def pull_CRSP_stock():
    """
    Pulls CRSP stock data from the WRDS database.

    Parameters:
    - wrds_username (str): WRDS username (default: WRDS_USERNAME)

    Returns:
    - crsp_monthly (pd.DataFrame): CRSP stock data

    """
    crsp_monthly_query = (
    "SELECT msf.permno, msf.permco, msf.date, "
            "date_trunc('month', msf.date)::date as month, "
            "msf.ret, msf.retx, msf.shrout, msf.altprc, "
            "msenames.exchcd, msenames.siccd, "
            "msedelist.dlret, msedelist.dlstcd "
        "FROM crsp.msf AS msf "
        "LEFT JOIN crsp.msenames as msenames "
        "ON msf.permno = msenames.permno AND "
        "msenames.namedt <= msf.date AND "
        "msf.date <= msenames.nameendt "
        "LEFT JOIN crsp.msedelist as msedelist "
        "ON msf.permno = msedelist.permno AND "
        "date_trunc('month', msf.date)::date = "
        "date_trunc('month', msedelist.dlstdt)::date "
        f"WHERE msf.date BETWEEN '{start_date}' AND '{end_date}' "
        "AND msenames.shrcd IN (10, 11)"
    )

In [100]:
def pull_compustat():
    """
    Pulls financial data from the Compustat database for a specified time period.

    Parameters:
    - wrds_username (str): The username for accessing the WRDS database. Defaults to WRDS_USERNAME.

    Returns:
    - compustat (DataFrame): A DataFrame containing the financial data from the Compustat database.

    """
    compustat_query = (
    "SELECT gvkey, datadate, seq, ceq, at, lt, txditc, txdb, itcb,  pstkrv, "
            "pstkl, pstk, capx, oancf, sale, cogs, xint, xsga, sich, ni, ebit, dp "
        "FROM comp.funda "
        "WHERE indfmt = 'INDL' "
            "AND datafmt = 'STD' "
            "AND consol = 'C' "
            f"AND datadate BETWEEN '{start_date}' AND '{end_date}'"
    )

Once we have these 2 tables, we need an additional one to be able to link them:

In [101]:
def pull_CRSP_Comp_Link_Table(crsp_monthly):
    """
    Pulls the CRSP Compustat Link Table from the WRDS database and merges it with the CRSP monthly data.

    Parameters:
    - wrds_username (str): The username for accessing the WRDS database.
    - crsp_monthly (pd.DataFrame): The CRSP monthly data.

    Returns:
    - pd.DataFrame: The merged data containing the CRSP Compustat Link Table.

    """

    ccmxpf_linktable_query = (
    "SELECT lpermno AS permno, gvkey, linkdt, "
            "COALESCE(linkenddt, CURRENT_DATE) AS linkenddt "
        "FROM crsp.ccmxpf_linktable "
        "WHERE linktype IN ('LU', 'LC') "
            "AND linkprim IN ('P', 'C') "
            "AND usedflag = 1"
    )

    ccmxpf_linktable = pd.read_sql_query(
    sql=ccmxpf_linktable_query,
    con=wrds,
    dtype={"permno": int, "gvkey": str},
    parse_dates={"linkdt", "linkenddt"}
    )
    ccm_links = (crsp_monthly
    .merge(ccmxpf_linktable, how="inner", on="permno")
    .query("~gvkey.isnull() & (date >= linkdt) & (date <= linkenddt)")
    .get(["permno", "gvkey", "date"])
    )

    crsp_monthly = (crsp_monthly
    .merge(ccm_links, how="left", on=["permno", "date"])
    )
    
    return crsp_monthly

After running these, the CRSP data should have the following variables

In [102]:
crsp_monthly = pd.read_parquet('../data/pulled/CRSP_stock.parquet')
crsp_monthly.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3420181 entries, 0 to 3481102
Data columns (total 14 columns):
 #   Column    Dtype         
---  ------    -----         
 0   permno    int64         
 1   permco    int64         
 2   date      datetime64[ns]
 3   month     datetime64[ns]
 4   ret       float64       
 5   retx      float64       
 6   shrout    float64       
 7   altprc    float64       
 8   exchcd    int64         
 9   siccd     int64         
 10  me        float64       
 11  me_lag    float64       
 12  exchange  object        
 13  ret_adj   float64       
dtypes: datetime64[ns](2), float64(7), int64(4), object(1)
memory usage: 391.4+ MB


permno and permco are unique indentifies for each company on the exchange. Date and month are straighforward. Further, ret is the monthly return of the equity, while retx is the return without dividends. shrout represents the number of outstading shares, while altprc gives the share price. exchcd takes an integer value that stands for the main exchange listed on (NASDAQ, NYSE, AMEX, or Other). Finally, siccd are SIC codes that also serve as identifiers.

The code for the other variables can be found below.

## mktcp
Calculating the monthly market equity for each firm using shares outstanding and share price. In addition, we also have a lagged market cap as some of the later calculations involve looking at the market equity of previous periods.

In [103]:
mktcap_lag = (crsp_monthly
    .assign(
        month=lambda x: x["month"]+pd.DateOffset(months=1),
        mktcap_lag=lambda x: x["me"]
    )
    .get(["permno", "month", "me_lag"])
    )

## exchange
This is simply hardcoding the exchange from the exchcd variable.
- NYSE had code 1 or 31
- AMEX has code 2 or 32
- NASDAQ has code 3 or 33

In [104]:
def assign_exchange(exchcd):
        
        # FILL IN THE CODE HERE

    crsp_monthly["exchange"] = (crsp_monthly["exchcd"]
        .apply(assign_exchange)
    )

Once the cleaning has been done to the CRSP data, the summary statistics should match the table below

In [105]:
crsp_monthly.describe()

,permno,permco,date,month,ret,retx,shrout,altprc,exchcd,siccd,me,me_lag,ret_adj
count,3.420181e+06,3.420181e+06,3420181,3420181,3.377429e+06,3.377429e+06,3.420179e+06,3.406788e+06,3.420181e+06,3.420181e+06,3.406667e+06,3.379253e+06,3.377434e+06
mean,5.297648e+04,1.796992e+04,1994-05-03 03:45:15.023678592,1994-04-04 04:37:35.598519168,1.127520e-02,1.001186e-02,5.042633e+07,3.494201e+01,2.249042e+00,4.728491e+03,2.115456e+03,2.114359e+03,1.124619e-02
min,1.000000e+04,3.000000e+00,1960-01-29 00:00:00,1960-01-01 00:00:00,-9.956900e-01,-9.956900e-01,0.000000e+00,-1.832500e+03,1.000000e+00,0.000000e+00,1.056250e-02,1.718750e-02,-1.000000e+00
25%,2.564600e+04,6.300000e+03,1982-05-28 00:00:00,1982-05-01 00:00:00,-6.730800e-02,-6.878800e-02,3.028000e+06,1.810000e+00,1.000000e+00,3.311000e+03,1.998862e+01,2.014950e+01,-6.730800e-02
50%,5.427900e+04,1.482000e+04,1994-07-29 00:00:00,1994-07-01 00:00:00,0.000000e+00,0.000000e+00,9.153000e+06,1.041000e+01,3.000000e+00,4.469000e+03,8.820000e+01,8.859967e+01,0.000000e+00
75%,7.973800e+04,2.312700e+04,2006-01-31 00:00:00,2006-01-01 00:00:00,7.142900e-02,7.002800e-02,3.007200e+07,2.487500e+01,3.000000e+00,6.311000e+03,4.930639e+02,4.946360e+02,7.142900e-02
max,9.343600e+04,5.975000e+04,2023-12-29 00:00:00,2023-12-01 00:00:00,2.400000e+01,2.400000e+01,2.920640e+10,5.467250e+05,3.300000e+01,9.999000e+03,3.071345e+06,3.071345e+06,2.400000e+01
std,2.768620e+04,1.488959e+04,NaN,NaN,1.839305e-01,1.839464e-01,2.689379e+08,2.138450e+03,9.155700e-01,2.228234e+03,1.957998e+04,1.944427e+04,1.839675e-01


# 3. Merging and Portfolio Assignment

Once we have all the data pulled down, we need to merge the CRSP and compustat data and begin calculating the appropriate metrics to assign the data to portfolios

### 1. Industry portfolios

Matching on SIC codes

In [106]:
def assign_industry5(sic_code):
    """
    Assigns an industry category based on the given SIC code.

    Parameters:
    sic_code (int): The SIC code to be categorized.

    Returns:
    str: The industry category assigned based on the SIC code range.
    """

    try:
        sic_code = int(sic_code)  # Ensure SIC code is an integer
    except ValueError:
        return 'Other'  # Return 'Other' if SIC code cannot be converted to integer

    # Define SIC code ranges for each industry
    cnsmr_ranges = [(100, 999), (2000, 2399), (2700, 2749), (2770, 2799), (3100, 3199),
                    (3940, 3989), (2500, 2519), (2590, 2599), (3630, 3659), (3710, 3711),
                    (3714, 3714), (3716, 3716), (3750, 3751), (3792, 3792), (3900, 3939),
                    (3990, 3999), (5000, 5999), (7200, 7299), (7600, 7699)]
    manuf_ranges = []
    hitec_ranges = []
    hlth_ranges = [] ## FILL IN MISSING CODE

    # Assign industry based on SIC code range
    for start, end in cnsmr_ranges:
        if start <= sic_code <= end:
            return 'Cnsmr'
            
            # Finish Assignment

### 2. Univariate sorts on E/P and C/P ratios

Need to calculate the breakpoints for each metric and then apply the appropriate grouping to each row. An example of both steps is found below. 

In [107]:
def categorize_stocks_by_metric(dataframe, metric, category_name):
    """
    Categorize stocks by a specified metric and directly append the categories to the dataframe.
    """
    # Calculate breakpoints for the NYSE stocks each year to define categories
    for year, group in dataframe.groupby('year'):
        non_negative = group[group[metric] >= 0]
        negative = group[group[metric] < 0]

        sorted_values = non_negative[metric].sort_values()

        bottom_30_bp = sorted_values.quantile(q=0.3)

        # Finish quintiles and deciles

        dataframe.loc[non_negative.index, category_name] = non_negative.apply(
            lambda row: categorize_metric_exclusive(row, metric, bottom_30_bp, top_30_bp, quintiles_bp.values, deciles_bp.values), axis=1)
        dataframe.loc[negative.index, category_name] = [['Negative Values']] * len(negative)

    return dataframe

In [108]:
def categorize_metric_exclusive(row, metric, bottom_30_bp, top_30_bp, quintiles_bp, deciles_bp):
    """
    Determine categories for a given metric value within a row.
    This is a placeholder function. Replace its internals based on actual categorization logic.
    """
    value = row[metric]
    categories = []

    if value < 0:
        categories.append('<=0')
    
    # COMPLETE

    quintiles = np.searchsorted(quintiles_bp, value, side='right')
    categories.append(f'Qnt {quintiles+1}')

    deciles = np.searchsorted(deciles_bp, value, side='right')
    categories.append(f'Dec {deciles+1}')

    return categories

### 3. Bivariate sorts on Operating Profitability and Investment

These metrics are already calculated within compustat, so the only goal here is to assign each observation to the appropriate quintile for each metric. The below assignment function can come in handy here.

In [109]:
def assign_portfolio(data, sorting_variable, n_portfolios):
    """Assign portfolio for a given sorting variable."""
    
    breakpoints = (data
      .get(sorting_variable)
      .quantile(np.linspace(0, 1, num=n_portfolios+1), 
                interpolation="linear")
      .drop_duplicates()
    )
    breakpoints.iloc[0] = -np.Inf
    breakpoints.iloc[breakpoints.size-1] = np.Inf
    
    assigned_portfolios = pd.cut(
      data[sorting_variable],
      bins=breakpoints,
      labels=range(1, breakpoints.size),
      include_lowest=True,
      right=False
    )
    
    return assigned_portfolios

From here, tables can be formed by grouping on the portfolio type(s) and calculating returns based on desired weightings (mainly value and equal weight).

<!-- ccm4['weight'] = ccm4['me'] / ccm4.groupby(['date', 'opport', 'invport'])['me'].transform('sum')
ccm4['weighted_ret'] = ccm4['retx'] * ccm4['weight']
vwret_m = ccm4.groupby(['date', 'opport', 'invport'])['weighted_ret'].sum().reset_index(name='value_weighted_ret')

ccm4['equal_weight'] = 1 / ccm4.groupby(['date', 'opport', 'invport'])['permno'].transform('count')
ccm4['equal_weighted_ret'] = ccm4['retx'] * ccm4['equal_weight']
ewret_m = ccm4.groupby(['date', 'opport', 'invport'])['equal_weighted_ret'].sum().reset_index(name='equal_weighted_ret') -->


GOOD LUCK!!